# Clip + Coop + TPT Delivery

*Group Composition*
- Emmanuele V. Coppola
- Ivan Doná
- Ettore Saggiorato

## Baseline

#### Installing necessary packages

In [1]:
!pip install -q ftfy regex tqdm scikit-learn scikit-image
!pip install -q git+https://github.com/openai/CLIP.git


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
  error: subprocess-exited-with-error
  
  × git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-62iros0r did not run successfully.
  │ exit code: 128
  ╰─> [1 lines of output]
      fatal: unable to access 'https://github.com/openai/CLIP.git/': Could not resolve host: github.com
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-62iros0r did not run successfully.
│ exit code: 128
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


#### Download and extract the datasets

In [2]:
!gdown --fuzzy https://drive.google.com/file/d/1nfictDVptrdDwRNaxsBx0UIlP7tpDsN_/view?usp=sharing

zsh:1: no matches found: https://drive.google.com/file/d/1nfictDVptrdDwRNaxsBx0UIlP7tpDsN_/view?usp=sharing


In [3]:
#Download metadata for class labelling
!curl https://raw.githubusercontent.com/modestyachts/ImageNetV2/refs/heads/master/data/metadata/class_info.json -o datasets/imagenetv2-matched-frequency-format-val/class_info.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: raw.githubusercontent.com


In [4]:
!mkdir datasets
!tar -xvf imagenet-a.tar -C datasets

mkdir: cannot create directory ‘datasets’: File exists


tar: imagenet-a.tar: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


In [5]:

import clip
import torch
import logging
import matplotlib.pyplot as plt
import numpy as np
import os
import gc
import sys
import skimage
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as v2
from pathlib import Path
from PIL import Image
from torch.utils.data import Dataset
from torch.utils.tensorboard import SummaryWriter
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize
from torchvision import transforms
from tqdm import tqdm
from torch.cuda.amp import autocast
from clip.simple_tokenizer import SimpleTokenizer as _Tokenizer
import json
import copy


from PIL import Image
import random

try:
    from torchvision.transforms import InterpolationMode
    BICUBIC = InterpolationMode.BICUBIC
except ImportError:
    BICUBIC = Image.BICUBIC

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

#### Utilities

In [6]:
#Reproducibility steps by https://docs.pytorch.org/docs/stable/notes/randomness.html
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

torch.manual_seed(42)
np.random.seed(42)

g = torch.Generator()
g.manual_seed(0)


def show_augmented_images(dataset, num_images=16, label="augmented"):
    """
    Show a grid of augmented images from the dataset.
    num_images must be a perfect square (e.g., 4, 9, 16, 25, 36, ...)
    """
    assert int(
        num_images ** 0.5) ** 2 == num_images, "num_images must be a perfect square"

    images = []
    for i in range(num_images):
        img_tensor, _ = dataset[i]
        images.append(img_tensor)

    # Make a grid of images
    grid = torchvision.utils.make_grid(
        images, nrow=int(num_images ** 0.5), padding=2)
    npimg = grid.numpy()

    # Convert and display
    plt.figure(figsize=(8, 8))
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.axis('off')
    plt.title(f'{num_images} Augmented Versions (including original) of {label}')
    plt.show()


def show_augmented_images_from_id_list(dataset, id_list, title=""):
    """
    Show a grid of augmented images from the dataset.
    num_images must be a perfect square (e.g., 4, 9, 16, 25, 36, ...)
    """

    images = []
    for i in id_list:
        img_tensor, _ = dataset[i]
        images.append(img_tensor)

    # Make a grid of images
    grid = torchvision.utils.make_grid(
        images, nrow=int(len(id_list) ** 0.5), padding=2)
    npimg = grid.numpy()

    # Convert and display
    plt.figure(figsize=(8, 8))
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.axis('off')
    plt.title(f'{len(id_list)} Augmented Versions: {title}')
    plt.show()


def show_tensor_images(tensor_images):
    """
    Show a grid of augmented images from the dataset.
    num_images must be a perfect square (e.g., 4, 9, 16, 25, 36, ...)
    """

    # Make a grid of images
    grid = torchvision.utils.make_grid(
        tensor_images, nrow=int(len(tensor_images) ** 0.5), padding=2)
    npimg = grid.numpy()

    # Convert and display
    plt.figure(figsize=(8, 8))
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.axis('off')
    plt.title(f'{len(tensor_images)} Augmented Versions')
    plt.show()


def LINE():
    return sys._getframe(1).f_lineno

In [7]:
def log_values(writer, step, loss, accuracy, prefix):
    writer.add_scalar(f"{prefix}/loss", loss, step)
    writer.add_scalar(f"{prefix}/accuracy", accuracy, step)

In [8]:
_tokenizer = _Tokenizer()
vis_net, basic_image_transformations = clip.load('ViT-B/16', DEVICE)

#### Dataset

In [9]:
class ImageNetA(Dataset):
    def __init__(self, root_dir='datasets/imagenet-a', transform=basic_image_transformations):
        self.root_dir = root_dir
        self.transform = transform

        # Load class code to name mapping from README.txt
        self.class_code_to_name = self._load_class_mapping(
            os.path.join(root_dir, 'README.txt'))

        # Map class codes to integer labels
        self.class_codes = sorted([
            d for d in os.listdir(root_dir)
            if os.path.isdir(os.path.join(root_dir, d)) and d in self.class_code_to_name
        ])
        self.class_code_to_idx = {code: idx for idx,
                                  code in enumerate(self.class_codes)}

        # Collect all image paths and labels
        self.samples = []
        for class_code in self.class_codes:
            class_folder = os.path.join(root_dir, class_code)
            for fname in os.listdir(class_folder):
                if fname.lower().endswith(('.png', '.jpg', '.jpeg')):
                    path = os.path.join(class_folder, fname)
                    label = self.class_code_to_idx[class_code]
                    self.samples.append((path, label))

    def _load_class_mapping(self, readme_path):
        mapping = {}
        with open(readme_path, 'r') as f:
            lines = f.readlines()[12:]  # Skip the first 12 lines
            for line in lines:
                parts = line.strip().split(' ', 1)
                if len(parts) == 2:
                    code, name = parts
                    mapping[code] = name
        return mapping

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        image_path, label = self.samples[idx]
        image = Image.open(image_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

In [10]:
class ImageNetV2(Dataset):
    def __init__(self, root_dir='datasets/imagenetv2-matched-frequency-format-val', transform=basic_image_transformations, use_imagenet_a_classes = True, imagenet_a = None):
        self.root_dir = root_dir
        self.transform = transform
        self.use_imagenet_a = use_imagenet_a_classes
        
        if use_imagenet_a_classes:
            assert type(imagenet_a) == ImageNetA, "imagenet_a_classes set to TRUE without passing imagenet_a object"
            imagenet_a_class_code_to_idx = imagenet_a.class_code_to_idx
            self.v2id_to_info = self._load_class_mapping(
                os.path.join(root_dir, 'class_info.json'), use_imagenet_a_classes, imagenet_a_class_code_to_idx)
            self.class_code_to_name = copy.deepcopy(imagenet_a.class_code_to_name)
            
        else:
            self.v2id_to_info = self._load_class_mapping(
                os.path.join(root_dir, 'class_info.json'), use_imagenet_a_classes,None)
            self.class_code_to_name = {
                idx: self.v2id_to_info["label"] for idx in self.v2id_to_info.keys()}
        

        
        self.samples = []
        for v2_class_code in self.v2id_to_info.keys():
            class_folder = os.path.join(root_dir, str(v2_class_code))
            for fname in os.listdir(class_folder):
                if fname.lower().endswith(('.png', '.jpg', '.jpeg')):
                    path = os.path.join(class_folder, fname)
                    if use_imagenet_a_classes:
                        self.samples.append((path, self.v2id_to_info[v2_class_code]["label_id"]))
                    else:
                        self.samples.append((path, v2_class_code))
        
    def _load_class_mapping(self, infofile_path, use_imagenet_a_classes, imagenet_a_class_code_to_idx:dict[str, int]):
        mapping = {}
        with open(infofile_path) as f:
            data = json.load(f)
            for idx,item in enumerate(data):
                if use_imagenet_a_classes:
                    if item["wnid"] in imagenet_a_class_code_to_idx.keys():
                        mapping[item["cid"]] = {"label_id": imagenet_a_class_code_to_idx[item["wnid"]],
                                                "ia_code": item["wnid"], "label": item["synset"][0].lower().replace(" ", "_")}
                else:
                    mapping[item["cid"]] = {"label":item["synset"][0].lower().replace(
                        " ", "_")}
        return mapping
    
    
    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        image_path, label = self.samples[idx]
        image = Image.open(image_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

In [11]:
def get_dataset_split(dataset, train_percentage=0.5, validation_percentage=0.25):
    # Load data

    # Create train validation and test samples
    num_samples = len(dataset)
    training_sample = int(num_samples * train_percentage + 1)
    validation_sample = int(num_samples * validation_percentage)
    test_sample = num_samples - training_sample - validation_sample

    training_dataset, validation_dataset, test_dataset = torch.utils.data.random_split(
        dataset, [training_sample, validation_sample, test_sample])

    return (training_dataset, validation_dataset, test_dataset)


def get_data(training_dataset, validation_dataset, test_dataset, batch_size=64, transform=None, num_workers=8):
    """
    Load the dataset, split it into train/val/test and return a DataLoader for each.
    """

    if not transform:
        transform = torchvision.transforms.Compose(
            [torchvision.transforms.ToTensor()])

    # Create a DataLoader
    train_loader = torch.utils.data.DataLoader(
        training_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers,    worker_init_fn=seed_worker,
        generator=g)
    val_loader = torch.utils.data.DataLoader(
        validation_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers,    worker_init_fn=seed_worker,
        generator=g)
    test_loader = torch.utils.data.DataLoader(
        test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers,    worker_init_fn=seed_worker,
        generator=g)

    return train_loader, val_loader, test_loader


def embed_dataset_classnames(dataset:ImageNetA, model, templates=["a photo of a {}."]):
    """
    Embed the classnames in the prompt template.
    Return the classnames and the normalized textual features.
    """
    # Create the list of descriptions and tokenize them
    classnames = dataset.class_code_to_name.values()

    texts_z_views = []
    for template in templates:
        descriptions = [template.format(c) for c in classnames]
        text_tokens = clip.tokenize(descriptions).to(DEVICE)

        # Get the normalized textual features
        with torch.no_grad():
            texts_z = model.encode_text(text_tokens).float()
            texts_z /= texts_z.norm(dim=-1, keepdim=True)
            texts_z_views.append(texts_z)

    # Evaluate the mean representation
    texts_z = torch.stack(texts_z_views).mean(dim=0)

    # Renormalise
    texts_z /= texts_z.norm(dim=-1, keepdim=True)

    return classnames, texts_z

#### Initialize datasets


In [12]:
dataset_a = ImageNetA()
dataset_v2 = ImageNetV2(imagenet_a=dataset_a)

### basic Clip

In [13]:
def new_model(model_class, dataset):
    classnames, _ = embed_dataset_classnames(
        dataset, vis_net)
    n_ctx = 4
    ctx_init = ""
    class_token_position = "end"
    csc = False
    coop_net = model_class(
        classnames=classnames, n_ctx=n_ctx, ctx_init=ctx_init, class_token_position=class_token_position, csc=csc
    ).to(DEVICE)
    return coop_net


def load_model(model):
    model.load_state_dict(torch.load(
        "./working_directory/model.pth", weights_only=True))
    model.eval()
    return model

In [14]:
class TextEncoder(nn.Module):
    def __init__(self, clip_model):
        super().__init__()
        self.transformer = clip_model.transformer
        self.positional_embedding = clip_model.positional_embedding
        self.ln_final = clip_model.ln_final
        self.text_projection = clip_model.text_projection

    def forward(self, prompts, tokenized_prompts):
        x = prompts + self.positional_embedding
        # [batch_size, n_ctx, transformer.width] -> [n_ctx, batch_size, transformer.width]
        x = x.permute(1, 0, 2)
        x = self.transformer(x)
        # [n_ctx, batch_size, transformer.width] -> [batch_size, n_ctx, transformer.width]
        x = x.permute(1, 0, 2)
        x = self.ln_final(x)

        # Take features from the eot embedding (eot_token is the highest number in each sequence)
        x = x[torch.arange(x.shape[0]), tokenized_prompts.argmax(
            dim=-1)] @ self.text_projection

        return x

In [15]:
from copy import deepcopy


# Basic mechanics are taken from the Lab Number 3 of 2024/2025 accademic year
class PromptLearner(nn.Module):
    def __init__(self, clip_model, classnames, n_ctx, ctx_init, class_token_position, csc=False):
        super().__init__()
        n_cls = len(classnames)
        ctx_dim = clip_model.ln_final.weight.shape[0]
        clip_imsize = clip_model.visual.input_resolution

        # Use given words to initialize context vectors
        if ctx_init:
            ctx_init = ctx_init.replace("_", " ")
            n_ctx = len(ctx_init.split(" "))
            prompt = clip.tokenize(ctx_init).to(
                clip_model.token_embedding.weight.device)
            with torch.no_grad():
                embedding = clip_model.token_embedding(prompt)
            ctx_vectors = embedding[0, 1: 1 + n_ctx, :]
            prompt_prefix = ctx_init
        else:
            if csc:
                print("Initializing class-specific contexts")
                ctx_vectors = torch.empty(n_cls, n_ctx, ctx_dim)
            else:
                print("Initializing a generic context")
                ctx_vectors = torch.empty(n_ctx, ctx_dim)

            torch.nn.init.normal_(ctx_vectors, std=0.02)
            prompt_prefix = " ".join(["X"] * n_ctx)

        print(f"Initial context: '{prompt_prefix}'")
        print(f"Number of context words (tokens): {n_ctx}")

        self.ctx_init_state = ctx_vectors.detach().clone()
        # These are the `prompts` we want to optimize
        self.ctx = nn.Parameter(ctx_vectors)

        print(classnames)
        classnames = [name.replace("_", " ") for name in classnames]
        name_lens = [len(_tokenizer.encode(name)) for name in classnames]
        prompts = [prompt_prefix + " " + name + "." for name in classnames]

        # print("+++")
        # print("Prompts:")
        # for p in prompts:
        #     print(p)
        # print("+++")

        tokenized_prompts = torch.cat([clip.tokenize(p) for p in prompts]).to(
            clip_model.token_embedding.weight.device)

        with torch.no_grad():
            embedding = clip_model.token_embedding(tokenized_prompts)

        # These token vectors will be saved when in save_model(),
        # but they should be ignored in load_model() as we want to use
        # those computed using the current class names
        self.register_buffer("token_prefix", embedding[:, :1, :])  # SOS
        self.register_buffer(
            "token_suffix", embedding[:, 1 + n_ctx:, :])  # CLS, EOS

        self.n_cls = n_cls
        self.n_ctx = n_ctx
        self.tokenized_prompts = tokenized_prompts
        self.name_lens = name_lens
        self.class_token_position = class_token_position
        self.ctx_checkpoint = ctx_vectors.detach().clone()

    def reset_ctx(self):  # https://discuss.pytorch.org/t/reset-model-weights/19180
        with torch.no_grad():
            self.ctx.copy_(self.ctx_checkpoint)

        self.ctx.requires_grad = True

    def set_ctx_checkpoint(self):
        with torch.no_grad():
            self.ctx_checkpoint.copy_(self.ctx)

    def forward(self):
        prefix = self.token_prefix
        suffix = self.token_suffix
        ctx = self.ctx

        # If CoOp, expand the ctx for all classes
        if ctx.dim() == 2:
            ctx = ctx.unsqueeze(0).expand(self.n_cls, -1, -1)

        if self.class_token_position == "end":
            prompts = torch.cat(
                [
                    prefix,  # (n_cls, 1, dim)
                    ctx,     # (n_cls, n_ctx, dim)
                    suffix,  # (n_cls, *, dim)
                ],
                dim=1,
            )

        elif self.class_token_position == "middle":
            half_n_ctx = self.n_ctx // 2
            prompts = []
            for i in range(self.n_cls):
                name_len = self.name_lens[i]
                prefix_i = prefix[i: i + 1, :, :]
                class_i = suffix[i: i + 1, :name_len, :]
                suffix_i = suffix[i: i + 1, name_len:, :]
                ctx_i_half1 = ctx[i: i + 1, :half_n_ctx, :]
                ctx_i_half2 = ctx[i: i + 1, half_n_ctx:, :]
                prompt = torch.cat(
                    [
                        prefix_i,     # (1, 1, dim)
                        ctx_i_half1,  # (1, n_ctx//2, dim)
                        class_i,      # (1, name_len, dim)
                        ctx_i_half2,  # (1, n_ctx//2, dim)
                        suffix_i,     # (1, *, dim)
                    ],
                    dim=1,
                )
                prompts.append(prompt)
            prompts = torch.cat(prompts, dim=0)

        elif self.class_token_position == "front":
            prompts = []
            for i in range(self.n_cls):
                name_len = self.name_lens[i]
                prefix_i = prefix[i: i + 1, :, :]
                class_i = suffix[i: i + 1, :name_len, :]
                suffix_i = suffix[i: i + 1, name_len:, :]
                ctx_i = ctx[i: i + 1, :, :]
                prompt = torch.cat(
                    [
                        prefix_i,  # (1, 1, dim)
                        class_i,   # (1, name_len, dim)
                        ctx_i,     # (1, n_ctx, dim)
                        suffix_i,  # (1, *, dim)
                    ],
                    dim=1,
                )
                prompts.append(prompt)
            prompts = torch.cat(prompts, dim=0)

        else:
            raise ValueError

        return prompts

In [16]:
import clip.model


class OurCLIP(nn.Module):
    def __init__(self, classnames, n_ctx, ctx_init, class_token_position, csc=False):
        super().__init__()
        clip_model: clip.model.CLIP
        clip_model, _ = clip.load("ViT-B/16")
        # clip_model = clip_model.cpu()
        clip_model = clip_model

        self.prompt_learner = PromptLearner(
            clip_model, classnames, n_ctx, ctx_init, class_token_position, csc=csc)
        self.tokenized_prompts = self.prompt_learner.tokenized_prompts
        self.image_encoder = clip_model.visual
        self.text_encoder = TextEncoder(clip_model)
        self.logit_scale = clip_model.logit_scale

    def reset_ctx(self):
        self.prompt_learner.reset_ctx()

    def set_ctx_checkpoint(self):
        self.prompt_learner.set_ctx_checkpoint()

    def forward(self, image):
        image_features = self.image_encoder(image)

        prompts = self.prompt_learner()
        tokenized_prompts = self.tokenized_prompts
        text_features = self.text_encoder(prompts, tokenized_prompts)

        image_features = image_features / \
            image_features.norm(dim=-1, keepdim=True)
        text_features = text_features / \
            text_features.norm(dim=-1, keepdim=True)

        logit_scale = self.logit_scale.exp()
        logits = logit_scale * image_features @ text_features.t()

        return logits

### COOP

In [17]:
def training_step(net: OurCLIP, data_loader: torch.utils.data.DataLoader, optimizer: torch.optim.Optimizer, cost_function, device=DEVICE):
    """
    Training step (for CoOp).
    """
    samples = 0.0
    cumulative_loss = 0.0
    cumulative_accuracy = 0.0

    # Set the network to training mode
    net.train()
    scaler = torch.cuda.amp.GradScaler(enabled=True)

    # Iterate over the training set
    pbar = tqdm(data_loader, desc="Training", position=0,
                leave=True, total=len(data_loader))
    for batch_idx, (inputs, targets) in enumerate(data_loader):
        # Load data into GPU
        inputs = inputs.to(device)
        """ print(f"input shape {inputs.shape}")
        print(f"input type {inputs.dtype}")
        print(f"labels{targets}") """

        targets = targets.to(device)

        # Forward pass
        with autocast():
            outputs = net(inputs)

        # Loss computation
        loss = cost_function(outputs, targets)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        # Gradients reset
        optimizer.zero_grad()

        # Fetch prediction and loss value
        samples += inputs.shape[0]
        cumulative_loss += loss.item()
        # max() returns (maximum_value, index_of_maximum_value)
        _, predicted = outputs.max(dim=1)

        # Compute training accuracy
        cumulative_accuracy += predicted.eq(targets).sum().item()

        pbar.set_postfix(train_loss=loss.item(),
                         train_acc=cumulative_accuracy / samples * 100)
        pbar.update(1)
        del inputs
        gc.collect()
        torch.cuda.empty_cache()

    return cumulative_loss / samples, cumulative_accuracy / samples * 100


def test_step(net, data_loader, cost_function, device="cuda"):
    samples = 0.0
    cumulative_loss = 0.0
    cumulative_accuracy = 0.0

    # Set the network to evaluation mode
    net.eval()

    # Disable gradient computation (we are only testing, we do not want our model to be modified in this step!)
    pbar = tqdm(data_loader, desc="Testing", position=0,
                leave=True, total=len(data_loader))
    with torch.no_grad():
        # Iterate over the test set
        for batch_idx, (inputs, targets) in enumerate(data_loader):
            # Load data into GPU
            inputs = inputs.to(device)
            targets = targets.to(device)

            # Forward pass
            with autocast():
                outputs = net(inputs)

            # Loss computation
            loss = cost_function(outputs, targets)

            # Fetch prediction and loss value
            samples += inputs.shape[0]
            # Note: the .item() is needed to extract scalars from tensors
            cumulative_loss += loss.item()
            _, predicted = outputs.max(1)

            # Compute accuracy
            cumulative_accuracy += predicted.eq(targets).sum().item()

            pbar.set_postfix(test_acc=cumulative_accuracy / samples * 100)
            pbar.update(1)

    return cumulative_loss / samples, cumulative_accuracy / samples * 100

In [18]:
def main_coop(
    net,
    dataset_splits,
    batch_size=16,
    learning_rate=0.002,
    weight_decay=0.0005,
    momentum=0.9,
    epochs=2,
    run_name="exp1",
    skip_test=False
):
    """
    @param: dataset_class
    @param: dataset_splits tuple that contains (training, validation, test) """

    # Create a logger for the experiment
    writer = SummaryWriter(log_dir=f"runs/{run_name}")

    # Get dataloaders
    train_loader, val_loader, test_loader = get_data(
        dataset_splits[0], dataset_splits[1], dataset_splits[2], transform=basic_image_transformations, batch_size=batch_size)

    print("Turning off gradients in both the image and the text encoder")
    for name, param in net.named_parameters():
        if "prompt_learner" not in name:
            param.requires_grad_(False)

    print(f"Total parameters: {sum(p.numel() for p in net.parameters()):,}")
    print(
        f"Total trainable parameters: {sum(p.numel() for p in net.parameters() if p.requires_grad):,}")

    # Instantiate the optimizer
    optimizer = torch.optim.SGD([
        {"params": net.parameters()}
    ], lr=learning_rate, weight_decay=weight_decay, momentum=momentum)

    # Define the cost function
    cost_function = torch.nn.CrossEntropyLoss()


    # Computes evaluation results before training
    if not skip_test:
        print("Before training:")
        train_loss, train_accuracy = test_step(
            net, train_loader, cost_function)
        val_loss, val_accuracy = test_step(net, val_loader, cost_function)
        test_loss, test_accuracy = test_step(net, test_loader, cost_function)

        # Log to TensorBoard
        log_values(writer, -1, train_loss, train_accuracy, "train")
        log_values(writer, -1, val_loss, val_accuracy, "validation")
        log_values(writer, -1, test_loss, test_accuracy, "test")

        print(
            f"\tTraining loss {train_loss:.5f}, Training accuracy {train_accuracy:.2f}")
        print(
            f"\tValidation loss {val_loss:.5f}, Validation accuracy {val_accuracy:.2f}")
        print(
            f"\tTest loss {test_loss:.5f}, Test accuracy {test_accuracy:.2f}")

    # For each epoch, train the network and then compute evaluation results
    for e in range(epochs):
        train_loss, train_accuracy = training_step(
            net, train_loader, optimizer, cost_function)
        val_loss, val_accuracy = test_step(net, val_loader, cost_function)

        log_values(writer, e, train_loss, train_accuracy, "train")
        log_values(writer, e, val_loss, val_accuracy, "validation")

    # Compute final evaluation results
    if not skip_test:
        print("After training:")

        train_loss, train_accuracy = test_step(
            net, train_loader, cost_function)
        val_loss, val_accuracy = test_step(net, val_loader, cost_function)
        test_loss, test_accuracy = test_step(net, test_loader, cost_function)

        log_values(writer, epochs, train_loss, train_accuracy, "train")
        log_values(writer, epochs, val_loss, val_accuracy, "validation")
        log_values(writer, epochs, test_loss, test_accuracy, "test")
        print(
            f"\tTraining loss {train_loss:.5f}, Training accuracy {train_accuracy:.2f}")
        print(
            f"\tValidation loss {val_loss:.5f}, Validation accuracy {val_accuracy:.2f}")
        print(
            f"\tTest loss {test_loss:.5f}, Test accuracy {test_accuracy:.2f}")

    # Closes the logger
    writer.close()
    return net

In [19]:
""" coop_net = new_model(OurCLIP, dataset_v2)
splitted_datasets = get_dataset_split(dataset_v2)
main_coop(coop_net, splitted_datasets, batch_size=16, skip_test = True)
torch.save(coop_net.state_dict(), "./working_directory/model_coop.pth") """

' coop_net = new_model(OurCLIP, dataset_v2)\nsplitted_datasets = get_dataset_split(dataset_v2)\nmain_coop(coop_net, splitted_datasets, batch_size=16, skip_test = True)\ntorch.save(coop_net.state_dict(), "./working_directory/model_coop.pth") '

### TPT

##### Augmix

In [20]:
# Example AugMix-style transform
def get_augmix_transform():
    return transforms.Compose([
        transforms.Resize(256),
        transforms.RandomResizedCrop(224, scale=(0.5, 1.0)),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(0.4, 0.4, 0.4),
        transforms.ToTensor(),
    ])


# Basic original transform (non-augmented)
original_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
])

# Wrapper Dataset that takes a base dataset + index of the sample to augment


class AugmentSingleSampleDataset(Dataset):
    def __init__(self, base_dataset, sample_idx, num_augments=63):
        self.base_dataset = base_dataset
        self.sample_idx = sample_idx
        self.num_augments = num_augments
        self.augmix_transform = get_augmix_transform()
        self.original_transform = original_transform

        # Extract the image once to avoid loading it 64 times
        image, label = self.base_dataset[self.sample_idx]
        if isinstance(image, torch.Tensor):
            self.image = transforms.ToPILImage()(image)
        else:
            self.image = image
        self.label = label

    def __len__(self):
        return self.num_augments + 1  # 63 augments + 1 original

    def __getitem__(self, idx):
        if idx == 0:
            image = self.original_transform(self.image)
        else:
            image = self.augmix_transform(self.image)
        return image, self.label

##### TPT Procedure

In [21]:
def select_confident_samples(logits, top_p):
    """
    Select the p-percentile of samples with lowest entropy, i.e. highest confidence.
    """
    assert 0 <= top_p < 1, "The value must be between 0 and 1"
    batch_entropy = -(logits.softmax(1) * logits.log_softmax(1)).sum(1)
    idx = torch.argsort(batch_entropy, descending=False)[
        :int(batch_entropy.size()[0] * top_p)]
    return logits[idx], idx


def compute_avg_entropy(outputs):
    """
    Compute marginal entropy of samples and return the average.
    """
    # Calculate probabilities from logits
    probs = outputs.softmax(dim=1)
    # To avoid log(0), clamp probabilities to a minimum value
    probs = probs.clamp(min=1e-9)
    entropy = -(probs * probs.log()).sum(dim=1)
    return entropy.mean()

In [22]:
def test_step_tpt(net, dataset, optimizer, optimizer_state_dict, log_writer, num_aug=63, batch_size=16):
    """
    @param net takes a OurClip model type
    """
    samples = 0.0
    cumulative_loss = 0.0
    cumulative_accuracy = 0.0

    # Set the network to evaluation mode
    net.eval()

    # Disable gradient computation (we are only testing, we do not want our model to be modified in this step!)
    pbar = tqdm(range(len(dataset)), desc="TPT_testing", position=0,
                leave=True, total=len(dataset))
    # Iterate over the indices of the test set
    try:
        for sample_idx in pbar:  # Iterate through indices
            net.reset_ctx()
            optimizer.load_state_dict(optimizer_state_dict)
            # print(f"after Reset{torch.cuda.mem_get_info()}")

            # Create augmented dataset for the current sample
            aug_data = AugmentSingleSampleDataset(
                dataset, sample_idx, num_augments=num_aug)  # Pass the base dataset and index

            # Create a DataLoader for the augmented samples of this single image
            aug_dataloader = torch.utils.data.DataLoader(
                aug_data, batch_size=batch_size, shuffle=False, worker_init_fn=seed_worker,
                generator=g)

            # Process the augmented images for this sample
            all_outputs = []
            for images, labels in aug_dataloader:
                try:
                    with autocast():
                        # print(f"size batch {len(images)}")
                        images = images.to(DEVICE)
                        # print(torch.cuda.mem_get_info(), LINE())
                        outputs = net(images)  # Use the provided net
                        # print(torch.cuda.mem_get_info(), LINE())
                        # cpu_outputs = outputs.to("cpu")
                        all_outputs.append(outputs)
                        # print(torch.cuda.mem_get_info(), LINE())

                        """ del images
                        del outputs
                        torch.cuda.empty_cache()
                        gc.collect()
                        print(torch.cuda.mem_get_info(), LINE()) """

                except:
                    torch.cuda.memory._dump_snapshot("my_snapshot.pickle")
                    raise

            # Get the original label for this sample
            original_image, target = dataset[sample_idx]
            original_image = original_image.unsqueeze(0).to(DEVICE)
            # print(torch.cuda.mem_get_info(), LINE())

            # Make target a tensor and move to device
            target = torch.tensor([target]).to(DEVICE)

            # Concatenate outputs from all batches for this sample
            all_outputs = torch.cat(all_outputs, dim=0)

            # Select confident samples and compute average entropy
            top_outputs, _ = select_confident_samples(all_outputs, 0.2)
            loss = compute_avg_entropy(top_outputs)
            # Loss computation
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            # Forward pass
            with autocast():
                outputs = net(original_image)

            # Fetch prediction and loss value
            samples += original_image.shape[0]
            # Note: the .item() is needed to extract scalars from tensors
            cumulative_loss += loss.item()
            _, predicted = outputs.max(1)

            # Compute accuracy
            cumulative_accuracy += predicted.eq(target).sum().item()

            pbar.set_postfix(test_acc=cumulative_accuracy / samples * 100)
            pbar.update(1)
    except:
        raise
    finally:
        del all_outputs
        del aug_data

    return cumulative_loss / samples, cumulative_accuracy / samples * 100

In [23]:
def tpt_test(net, dataset:Dataset, run_name="tpt1", num_aug=63, batch_size=64):

    # Create a logger for the experiment
    log_writer = SummaryWriter(log_dir=f"runs/{run_name}")
    net.set_ctx_checkpoint()

    for name, param in net.named_parameters():
        if "prompt_learner" not in name:
            param.requires_grad_(False)
        # print(f"{name}is in {param.requires_grad}")
    print(torch.cuda.mem_get_info(), LINE())

    # Define the optimizer
    # optimizer = get_optimizer(net, 0.002, 0.0005, 0.9)
    # , weight_decay=wd, momentum=momentum)
    optimizer = torch.optim.AdamW([{"params": net.parameters()}], lr=0.005)
    optimizer_state_dict = deepcopy(optimizer.state_dict())
    print(torch.cuda.mem_get_info(), LINE())

    print("Test tpt:")
    test_loss, test_accuracy = test_step_tpt(
        net, dataset, optimizer, optimizer_state_dict, log_writer, num_aug=num_aug, batch_size=batch_size)

    # Closes the logger
    log_writer.close()

In [24]:
coop_net = new_model(OurCLIP, dataset_a)
coop_net = load_model(coop_net)

Initializing a generic context
Initial context: 'X X X X'
Number of context words (tokens): 4
dict_values(['stingray', 'goldfinch', 'junco', 'American robin', 'jay', 'bald eagle', 'vulture', 'newt', 'American bullfrog', 'box turtle', 'green iguana', 'agama', 'chameleon', 'American alligator', 'garter snake', 'harvestman', 'scorpion', 'tarantula', 'centipede', 'sulphur-crested cockatoo', 'lorikeet', 'hummingbird', 'toucan', 'duck', 'goose', 'koala', 'jellyfish', 'sea anemone', 'flatworm', 'snail', 'crayfish', 'hermit crab', 'flamingo', 'great egret', 'oystercatcher', 'pelican', 'sea lion', 'Chihuahua', 'Golden Retriever', 'Rottweiler', 'German Shepherd Dog', 'pug', 'red fox', 'Persian cat', 'lynx', 'lion', 'American black bear', 'mongoose', 'ladybug', 'rhinoceros beetle', 'weevil', 'fly', 'bee', 'ant', 'grasshopper', 'stick insect', 'cockroach', 'mantis', 'leafhopper', 'dragonfly', 'monarch butterfly', 'small white', 'gossamer-winged butterfly', 'starfish', 'cottontail rabbit', 'porcupi

In [25]:
tpt_test(coop_net, dataset_a, batch_size=64, num_aug=63)

(6794510336, 8212709376) 11
(6794510336, 8212709376) 18
Test tpt:


TPT_testing: 100%|██████████| 7500/7500 [1:25:34<00:00,  1.46it/s, test_acc=55.2]


## Tests

### Film

In [26]:
class FiLMModulation(nn.Module):
    def __init__(self, text_dim, image_dim):
        super().__init__()
        self.film = nn.Sequential(
            nn.Linear(text_dim, image_dim * 2),
            nn.ReLU()
        )

    def forward(self, image_feat, text_feat):
        gamma_beta = self.film(text_feat)  # [B, 2 * C]
        gamma, beta = gamma_beta.chunk(2, dim=-1)  # [B, C] each
        return gamma * image_feat + beta


class FilmCLIP(nn.Module):
    def __init__(self, classnames, n_ctx, ctx_init, class_token_position, csc=False):
        super().__init__()
        clip_model, _ = clip.load("ViT-B/16")

        self.prompt_learner = PromptLearner(
            clip_model, classnames, n_ctx, ctx_init, class_token_position, csc=csc)
        self.tokenized_prompts = self.prompt_learner.tokenized_prompts
        self.image_encoder = clip_model.visual
        self.text_encoder = TextEncoder(clip_model)
        self.logit_scale = clip_model.logit_scale

        self.film_modulation = FiLMModulation(
            text_dim=512, image_dim=512)  # 512 is CLIP's embedding dim

    def reset_ctx(self):
        self.prompt_learner.reset_ctx()

    def set_ctx_checkpoint(self):
        self.prompt_learner.set_ctx_checkpoint()

    def forward(self, image):
        image_features = self.image_encoder(image)  # [B, 512]

        prompts = self.prompt_learner()
        tokenized_prompts = self.tokenized_prompts
        text_features = self.text_encoder(
            prompts, tokenized_prompts)  # [K, 512]

        # Average text features to get a global modulation vector
        """  global_text_feat = text_features.mean(dim=0, keepdim=True)  # [1, 512]
        global_text_feat = global_text_feat.expand(
            image_features.size(0), -1)  # [B, 512] """

        # Cosine similarity between each image and each class text feature
        sim = image_features @ text_features.t()  # [B, K]

        # Softmax over classes to get weights
        weights = sim.softmax(dim=1)  # [B, K]

        # Weighted average of text features per image
        global_text_feat = weights @ text_features  # [B, 512]

        # Modulate image features
        image_features = self.film_modulation(image_features, global_text_feat)

        # Normalize
        image_features = image_features / \
            image_features.norm(dim=-1, keepdim=True)
        text_features = text_features / \
            text_features.norm(dim=-1, keepdim=True)

        logit_scale = self.logit_scale.exp()
        logits = logit_scale * image_features @ text_features.t()

        return logits

In [ ]:
def main_film_coop(
    net,
    dataset_splits,
    batch_size=16,
    learning_rate=0.002,
    weight_decay=0.0005,
    momentum=0.9,
    epochs=2,
    run_name="exp1",
    skip_test=False
):
    """
    @param: dataset_class
    @param: dataset_splits tuple that contains (training, validation, test) """

    # Create a logger for the experiment
    writer = SummaryWriter(log_dir=f"runs/{run_name}")

    # Get dataloaders
    train_loader, val_loader, test_loader = get_data(
        dataset_splits[0], dataset_splits[1], dataset_splits[2], transform=basic_image_transformations, batch_size=batch_size)

    print("Turning off gradients in both the image and the text encoder adn film Module")
    for name, param in net.named_parameters():
        if "prompt_learner" not in name:
            param.requires_grad_(False)

    print(f"Total parameters: {sum(p.numel() for p in net.parameters()):,}")
    print(
        f"Total trainable parameters: {sum(p.numel() for p in net.parameters() if p.requires_grad):,}")

    # Instantiate the optimizer
    optimizer = get_optimizer(net, learning_rate, weight_decay, momentum)
ImageNetA
    # Define the cost function
    cost_function = get_cost_function()

    # Computes evaluation results before training
    if not skip_test:
        print("Before training:")
        train_loss, train_accuracy = test_step(
            net, train_loader, cost_function)
        val_loss, val_accuracy = test_step(net, val_loader, cost_function)
        test_loss, test_accuracy = test_step(net, test_loader, cost_function)

        # Log to TensorBoard
        log_values(writer, -1, train_loss, train_accuracy, "train")
        log_values(writer, -1, val_loss, val_accuracy, "validation")
        log_values(writer, -1, test_loss, test_accuracy, "test")

        print(
            f"\tTraining loss {train_loss:.5f}, Training accuracy {train_accuracy:.2f}")
        print(
            f"\tValidation loss {val_loss:.5f}, Validation accuracy {val_accuracy:.2f}")
        print(
            f"\tTest loss {test_loss:.5f}, Test accuracy {test_accuracy:.2f}")

    # For each epoch, train the network and then compute evaluation results
    for e in range(epochs):
        train_loss, train_accuracy = training_step(
            net, train_loader, optimizer, cost_function)
        val_loss, val_accuracy = test_step(net, val_loader, cost_function)

        log_values(writer, e, train_loss, train_accuracy, "train")
        log_values(writer, e, val_loss, val_accuracy, "validation")

    print("Turning off gradients prompt learner and activating the FILM ones ")
    for name, param in net.named_parameters():
        if "film_modulation" in name:
            param.requires_grad_(True)
            print(f"film grad {param.requires_grad}")
        else:
            param.requires_grad_(False)

    for e in range(epochs):
        train_loss, train_accuracy = training_step(
            net, train_loader, optimizer, cost_function)
        val_loss, val_accuracy = test_step(net, val_loader, cost_function)

        log_values(writer, e, train_loss, train_accuracy, "train")
        log_values(writer, e, val_loss, val_accuracy, "validation")

    # Compute final evaluation results
    print("After training:")

    train_loss, train_accuracy = test_step(
        net, train_loader, cost_function)
    val_loss, val_accuracy = test_step(net, val_loader, cost_function)
    test_loss, test_accuracy = test_step(net, test_loader, cost_function)

    log_values(writer, epochs, train_loss, train_accuracy, "train")
    log_values(writer, epochs, val_loss, val_accuracy, "validation")
    log_values(writer, epochs, test_loss, test_accuracy, "test")
    print(
        f"\tTraining loss {train_loss:.5f}, Training accuracy {train_accuracy:.2f}")
    print(
        f"\tValidation loss {val_loss:.5f}, Validation accuracy {val_accuracy:.2f}")
    print(
        f"\tTest loss {test_loss:.5f}, Test accuracy {test_accuracy:.2f}")

    # Closes the logger
    writer.close()
    return net

In [28]:
""" coop_film_net, preprocess = new_model(FilmCLIP)
splitted_datasets = get_dataset_split(ImageNetA)
coop_film_net.load_state_dict(torch.load(
    "./working_directory/film_model.pth", weights_only=True))
coop_film_net.eval() """

""" main_film_coop(coop_film_net, splitted_datasets, batch_size=16, skip_test=True)
torch.save(coop_film_net.state_dict(), "./working_directory/film_model.pth") """

# tpt_test(coop_film_net, splitted_datasets, batch_size=64, num_aug=63)

' main_film_coop(coop_film_net, splitted_datasets, batch_size=16, skip_test=True)\ntorch.save(coop_film_net.state_dict(), "./working_directory/film_model.pth") '

### Average Vision

In [29]:

def test_avg_vis_step_tpt(net: OurCLIP, dataset, optimizer, optimizer_state_dict, log_writer, num_aug=63, batch_size=16):
    """
    @param net takes a OurClip model type
    """
    samples = 0.0
    cumulative_loss = 0.0
    cumulative_accuracy = 0.0

    # Set the network to evaluation mode
    net.eval()

    # Disable gradient computation (we are only testing, we do not want our model to be modified in this step!)
    pbar = tqdm(range(len(dataset)), desc="TPT_avg_vis_testing", position=0,
                leave=True, total=len(dataset))
    # Iterate over the indices of the test set
    try:
        for sample_idx in pbar:  # Iterate through indices
            net.reset_ctx()
            optimizer.load_state_dict(optimizer_state_dict)
            # print(f"after Reset{torch.cuda.mem_get_info()}")

            # Create augmented dataset for the current sample
            aug_data = AugmentSingleSampleDataset(
                dataset.dataset, sample_idx, num_augments=num_aug)  # Pass the base dataset and index

            # Create a DataLoader for the augmented samples of this single image
            aug_dataloader = torch.utils.data.DataLoader(
                aug_data, batch_size=batch_size, shuffle=False, worker_init_fn=seed_worker,
                generator=g)

            # Process the augmented images for this sample
            all_outputs = []
            for images, labels in aug_dataloader:
                try:
                    with autocast():
                        images = images.to(DEVICE)
                        outputs = net(images)  # Use the provided net
                        all_outputs.append(outputs)

                except:
                    torch.cuda.memory._dump_snapshot("my_snapshot.pickle")
                    raise

            # Get the original label for this sample
            original_image, target = dataset[sample_idx]
            original_image = original_image.unsqueeze(0).to(DEVICE)

            # Make target a tensor and move to device
            target = torch.tensor([target]).to(DEVICE)

            # Concatenate outputs from all batches for this sample
            all_outputs = torch.cat(all_outputs, dim=0)

            # Select confident samples and compute average entropy
            top_outputs, top_image_idx = select_confident_samples(
                all_outputs, 0.2)
            loss = compute_avg_entropy(top_outputs)
            # Loss computation
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            # Building average embedding
            """ aug_dataloader = torch.utils.data.DataLoader(
                dataset.dataset[top_image_idx], batch_size=batch_size, shuffle=False, worker_init_fn=seed_worker,
                generator=g) """

            visual_outputs = []
            images = []
            for idx in top_image_idx:
                image, labels = aug_data[idx]
                images.append(image)
            try:
                images = torch.stack(images).to(DEVICE)

                with autocast():
                    visual_outputs.append(net.image_encoder(
                        images))  # Use the provided net

            except:
                torch.cuda.memory._dump_snapshot("my_snapshot.pickle")
                raise

            average_visual = torch.stack(visual_outputs).mean(dim=0)
            # Forward pass
            with autocast():
                prompts = net.prompt_learner()
                tokenized_prompts = net.tokenized_prompts
                text_features = net.text_encoder(
                    prompts, tokenized_prompts)

                image_features = average_visual / \
                    average_visual.norm(dim=-1, keepdim=True)
                text_features = text_features / \
                    text_features.norm(dim=-1, keepdim=True)

                logit_scale = net.logit_scale.exp()
                logits = logit_scale * image_features @ text_features.t()
                # outputs = net(original_image)

            # Fetch prediction and loss value
            samples += original_image.shape[0]
            # Note: the .item() is needed to extract scalars from tensors
            cumulative_loss += loss.item()
            _, predicted = logits.max(1)

            # Compute accuracy
            cumulative_accuracy += predicted.eq(target).sum().item()

            pbar.set_postfix(test_acc=cumulative_accuracy / samples * 100)
            pbar.update(1)
    except:
        raise
    finally:
        del all_outputs
        del aug_data

    return cumulative_loss / samples, cumulative_accuracy / samples * 100

In [30]:
def tpt_avg_vis_test(net, datasets, run_name="tpt_vis", num_aug=63, batch_size=64):

    # Create a logger for the experiment
    log_writer = SummaryWriter(log_dir=f"runs/{run_name}")
    net.set_ctx_checkpoint()

    for name, param in net.named_parameters():
        if "prompt_learner" not in name:
            param.requires_grad_(False)
        # print(f"{name}is in {param.requires_grad}")
    print(torch.cuda.mem_get_info(), LINE())

    # Define the optimizer
    # optimizer = get_optimizer(net, 0.002, 0.0005, 0.9)
    # , weight_decay=wd, momentum=momentum)
    optimizer = torch.optim.AdamW([{"params": net.parameters()}], lr=0.005)
    optimizer_state_dict = deepcopy(optimizer.state_dict())
    print(torch.cuda.mem_get_info(), LINE())

    print("Test tpt:")
    test_loss, test_accuracy = test_avg_vis_step_tpt(
        net, datasets[2], optimizer, optimizer_state_dict, log_writer, num_aug=num_aug, batch_size=batch_size)

    # Closes the logger
    log_writer.close()

In [31]:
""" coop_vis_net, preprocess = new_model(OurCLIP, ImageNetA)
splitted_datasets = get_dataset_split(ImageNetA)
coop_vis_net.load_state_dict(torch.load(
    "./working_directory/model.pth", weights_only=True))
coop_vis_net.eval()
tpt_avg_vis_test(coop_vis_net, splitted_datasets) """

' coop_vis_net, preprocess = new_model(OurCLIP, ImageNetA)\nsplitted_datasets = get_dataset_split(ImageNetA)\ncoop_vis_net.load_state_dict(torch.load(\n    "./working_directory/model.pth", weights_only=True))\ncoop_vis_net.eval()\ntpt_avg_vis_test(coop_vis_net, splitted_datasets) '